### 5-1. Temporal for many Features

objective
- temporal rotational velocity for Head, Finger, Combined eye, Combined EFRC
- plot with 95% confidence graph
- for eyes, filter values before application of mean

In [ ]:
import os, csv, pickle, gzip, torch, copy
import torch
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy
from scipy.spatial.transform import Rotation as R

from tqdm import tqdm
from matplotlib import rc #change xtick & ytick size
%matplotlib notebook
%matplotlib inline
matplotlib.rcParams['font.family'] = "Arial"
from scipy.signal import butter, lfilter, freqz


#========= read =========================================================================
# features(ffile):   return [trimX, trimY, trimAnimTs, trimGTPos, trimHmdTs, trimHmd1Ts, trimEyeTs, self.l_trim_idx]
# sorted_params(sfile):  (row=subject) (column=file_num)
# df_files(dffile): (param df data for all subject, file pairs)
# 	sub_idx	file_idx	is_exclude	start_idx	end_idx	length	h	v	d	x	y	z

def read_gzip(path):    
    # All VERSIONs should match (pickle, python, pandas)
    output = []
    assert os.path.exists(path), 'ReadGzip Assertion: path is null: {0}'.format(path)
    with gzip.open(path, 'rb') as file:
        print("open gzip")
        output = pickle.load(file)     
    return output

spath = "data/data_feature/sorted-params.csv"
sfile = open(spath, 'r', encoding='utf-8')
sfile = list(csv.reader(sfile))

# ver2 file makes vector feature velocity to hold a and b instead of simple subtraction that is useless
file_name = "0919-f18r16ver2"
fpath = "data/data_feature/" + file_name + ".gz"
ffile = read_gzip(fpath)

dfpath = "data/data_feature/0216-df.csv"
dffile = pd.read_csv(dfpath, na_values=" NaN").dropna(how='all', axis=1)
dffile = dffile[['sub_idx','file_idx','length', 'h','v','d']]
# print(dffile.head)

params_df = dffile[['h','v','d']]
params_tensor = torch.tensor(params_df.values, dtype=torch.float32)
PARAMS = params_tensor.reshape(24, 144, 3)
# print(PARAMS[0])


#=========== group ==================
#depth = (0.9, 1.7, 2.5, 3.3)
#angles = (+-30 azimuth, +-20 elevation)

# 2 5 8
# 1 4 7
# 0 3 6

#cross: 1,3,5,7
#center: 4
#oblique: 0,2,6,8

In [ ]:
#X for subject= [144, 18, 16]. For the latter half velocity section
#===========================================
#(HMD, lhand, rhand, index) 
#(e1, e2) **e0 was combined eye and not included 
#(efrch, efrc1, efrc2) 
#(vecprod: e1*efrch, e2*efrch, e1*efrc1, e2*efrc2)
#===========================================
def plot_velocity_profile_per_target(ffile, sfile, dffile, PARAMS):
    """
    FUNC: make 16 subject df list on velocity
    FUNC: make 36 combo df list on velocity
    """
    
    # ==== Title & condition flags ===============
    idx_to_plot = int(input("idx to plot"))
    features = ['HMD', 'RHand', 'RIndex', 'LEye', 'REye', 'HFRC', 'EFRC1', 'EFRC2', 'VECPROD']
    feature = features[idx_to_plot]
    flags = ['transform', 'vector', 'vecprod']
    flag = flags[0]
    if(idx_to_plot > 2):
        flag = flags[1]
    elif(idx_to_plot > 6):
        flag = flags[2]
    
    
#     #1. ====================
#     X_sub_comb_list = [] #[16][36][4]
#     for sIdx in range(16):
#         X = ffile[sIdx][0] #[144][F,18,16] X is already trimmed=============
#         pIdx = int(sfile[sIdx][fIdx]) #pointer idx
#         param = PARAMS[sIdx][pIdx]        
        
#         X_list_file = []
#         for fIdx in range(144): 
#             X_f = X[pIdx][:, 9:] #[F,9,16] 
#             X_list_file.append(X_f)
            
#         #======== package to (36 combination, 4 repeat) listlist
#         X_list_comb = [] 
#         for fIdx in range(36): #4repeats for 36 combo = 144 files
#             arr_idx = (cIdx, 36+cIdx, 72+cIdx, 108+cIdx)
#             X_comb = [X_list_file[i] for i in array_idx]
#             X_list_comb.append(X_comb)
            
#         X_sub_comb_list.append(X_list_comb)
            
    # ==== data per target ==============================
    X_comb_sub_list = [] #[36][64, 9, 9, 16] : ONLY VELOCITY
    X_comb_sub_list_pos = [] #[35][64, 9, 9, 16] : ONLY POSITION
    for cIdx in range(36):
        
        X_sub_list = []#64 = 16*4 files
        X_sub_list = []
        for sIdx in range(16):
            # Input for subject & pointer file index
            X_sub = ffile[sIdx][0] #[144][F,J,16]
            arr_pIdx0 = int(sfile[sIdx][cIdx]) 
            arr_pIdx1 = int(sfile[sIdx][cIdx+36]) 
            arr_pIdx2 = int(sfile[sIdx][cIdx+72]) 
            arr_pIdx3 = int(sfile[sIdx][cIdx+108])
            
            # 4 repeat files & velocity feature subset
            X_sub_list.append( X_sub[arr_pIdx0][:, 9:] )
            X_sub_list.append( X_sub[arr_pIdx1][:, 9:] )
            X_sub_list.append( X_sub[arr_pIdx2][:, 9:] )
            X_sub_list.append( X_sub[arr_pIdx3][:, 9:] )
                        
        
        # nest & pad list[tensors]
        X_sub_list = torch.nested.nested_tensor(X_sub_list) # [64, variable F, 16]
        X_sub_list = torch.nested.to_padded_tensor(X_sub_list, 0) # [64, common F, 16]
        X_comb_sub_list.append(X_sub_list)
        

    # ==== common plot ==============================
    fig = plt.figure(figsize=(25,10))
    plt.suptitle("Velocity Profile for " + feature, fontsize=18)
    matplotlib.rc('xtick', labelsize=15) 
    matplotlib.rc('ytick', labelsize=15)     
    ax0 = plt.subplot(2, 2, 1)
    leg = [*range(36)]
    max_y_val_angular = 0
    

    # ==== plot 1 ==============================
    ax1 = plt.subplot(231)
    ax1.set_title("Linear Velocity", fontsize=15)
    ax1.set_xlabel('time (sec)', fontsize=15)
    ax1.set_ylabel('m/s', fontsize=15) 
    # ax1.locator_params(nbins=6, axis='y') #xtick density 
    ax1.set_xlim(0,3)
    # ax1.set_ylim(0,0.35)  
    ax1.locator_params(nbins=5, axis='x') #xtick density

    # ==== plot 2 ==============================
    ax2 = plt.subplot(2, 3, 2)   
    ax2.set_title("Rotational Velocity", fontsize=15)
    ax2.set_xlabel('time (sec)', fontsize=15)
    ax2.set_ylabel('deg/s', fontsize=15)
    ax2.locator_params(nbins=5, axis='x') #xtick density    
    # ax2.locator_params(nbins=6, axis='y') #xtick density        
    ax2.set_xlim(0,3)
    ax2.set_ylim(-0.1,0.35)  
    
    # handles = [*str(range(36))]
    # ax2.legend(handles)
    
    # ==== plot 3 ==============================
    ax3 = plt.subplot(2, 3, 4)
    ax3.set_title("Sagittal Rotational Velocity", fontsize=15)
    ax3.set_xlabel('time (sec)', fontsize=15)
    ax3.set_ylabel('deg/s', fontsize=15) 
    # ax3.locator_params(nbins=6, axis='y') #xtick density 
    ax3.set_xlim(0,3)
    ax3.locator_params(nbins=5, axis='x') #xtick density

    
    # ==== plot 4 ==============================
    ax4 = plt.subplot(2, 3, 5)
    ax4.set_title("Horizontal Rotational Velocity", fontsize=15)
    ax4.set_xlabel('time (sec)', fontsize=15)
    ax4.set_ylabel('deg/s', fontsize=15) 
    # ax4.locator_params(nbins=6, axis='y') #xtick density 
    ax4.set_xlim(0,3)
    ax4.locator_params(nbins=5, axis='x') #xtick density

    # ==== plot 5 ==============================
    ax5 = plt.subplot(2, 3, 6)
    ax5.set_title("Coronal Rotational Velocity", fontsize=15)
    ax5.set_xlabel('time (sec)', fontsize=15)
    ax5.set_ylabel('deg/s', fontsize=15) 
    # ax5.locator_params(nbins=6, axis='y') #xtick density 
    ax5.set_xlim(0,3)
    ax5.locator_params(nbins=5, axis='x') #xtick density
    
    handle1=[]
    handle2=[]
    handle3=[]
    handle4=[]
    handle5=[]
    
    #==== grid plot for all axes ========
    major_ticks = np.arange(0, 5, 0.5)
    minor_ticks = np.arange(0, 5, 0.1)
    ax1.set_xticks(major_ticks)
    ax1.set_xticks(minor_ticks, minor=True)
    ax1.set_yticks(major_ticks)
    ax1.set_yticks(minor_ticks, minor=True)
    ax1.grid(which='both')    
    ax2.set_xticks(major_ticks)
    ax2.set_xticks(minor_ticks, minor=True)
    ax2.set_yticks(major_ticks)
    ax2.set_yticks(minor_ticks, minor=True)
    ax2.grid(which='both')   
    
    for cIdx in range(36):
        # Color for different configs
        
        #========COLOR PER DISTANCE ===========
        
        #========COLOR PER ANGLE ==============
        
        
        
        # ===== average data & downsample =========================
        X_comb = X_comb_sub_list[cIdx] #[64, F, 9, 16]
        X_comb_avg = torch.mean(X_comb, 0) #[F, 9, 16]
        X_plot = X_comb_avg[:, int(idx_to_plot)]
        frametime = 1/120
        
        # ===== calc lin/rot delta =========================
        if flag == 'transform':
            positions = X_plot[:,(3,7,11)]
            rot_matrix_list = torch.reshape(X_plot[:,(0,1,2, 4,5,6, 8,9,10)], (-1,3,3)) # MATRIX
            r = R.from_matrix(rot_matrix_list) #[N,3,3]
            rot_omega_list = calc_rot_omega_list(rot_matrix_list)
            
        elif flag == 'vector':         
            # https://gist.github.com/aormorningstar/3e5dda91f155d7919ef6256cb057ceee
            # finding a unique rotation is impossible with Kabsch algorithm
            positions = X_plot[:,(8,9,10)] #Trash values
            past_vec = X_plot[:,(0,1,2)] #Trash values
            curr_vec = X_plot[:,(4,5,6)] #Trash values
            rot_matrix_list = calc_rot_matrix_list_from_vecs(past_vec, curr_vec) # MATRIX
            r = R.from_matrix(rot_matrix_list)
            rot_omega_list = calc_rot_omega_list(rot_matrix_list)
            
        r_vec = torch.tensor(R.as_rotvec(r,  degrees=True)) #[N, 3]
        r_delta_angle = torch.linalg.vector_norm(r_vec, dim=1) #[N] #LOSS of DIRECTIONAL INFORMATION
        rot_omega_list = torch.tensor(rot_omega_list)
        r_delta_angle_x = rot_omega_list[:,2,1] #omega_x * dt
        r_delta_angle_y = rot_omega_list[:,0,2] #omega_y * dt
        r_delta_angle_z = rot_omega_list[:,1,0] #omega_Z * dt

        # ===== calc lin/rot velocity =========================
        y_linvel = torch.norm(positions, dim=1) / frametime
        y_rotvel = (r_delta_angle)/ frametime    
        y_rotvel_x = r_delta_angle_x / frametime
        y_rotvel_y = r_delta_angle_y / frametime
        y_rotvel_z = r_delta_angle_z / frametime
        
        #=======================================================
        # ===== Apply LPF for eye data with high frequency =====
        #=======================================================
        if idx_to_plot == 3 or idx_to_plot == 4:
            cutoff = 3 #Hz
            fs = 120 #Hz sampling
            order = 6
            y_rotvel = butter_lowpass_filter(y_rotvel, cutoff, fs, order)          
            y_rotvel_x = butter_lowpass_filter(y_rotvel_x, cutoff, fs, order)          
            y_rotvel_y = butter_lowpass_filter(y_rotvel_y, cutoff, fs, order)          
            y_rotvel_z = butter_lowpass_filter(y_rotvel_z, cutoff, fs, order)          
   
        cutoff = 3 #Hz
        fs = 120 #Hz sampling
        order = 6
        y_linvel = butter_lowpass_filter(y_linvel, cutoff, fs, order)    
        y_rotvel = butter_lowpass_filter(y_rotvel, cutoff, fs, order)          
        y_rotvel_x = butter_lowpass_filter(y_rotvel_x, cutoff, fs, order)          
        y_rotvel_y = butter_lowpass_filter(y_rotvel_y, cutoff, fs, order)          
        y_rotvel_z = butter_lowpass_filter(y_rotvel_z, cutoff, fs, order)          
           
        # ==== downsample & set x linspace ==========================
        new_frametime = frametime*3 #resolution should be at least 60Hz
        y_linvel = y_linvel[::3]
        y_rotvel = y_rotvel[::3]    
        y_rotvel_x = y_rotvel_x[::3]    
        y_rotvel_y = y_rotvel_y[::3]    
        y_rotvel_z = y_rotvel_z[::3]    
        x = torch.linspace(0, y_linvel.shape[0]*new_frametime, y_linvel.shape[0])
        
        # ===== plot linvel =========================
        line_1, = ax1.plot(x, y_linvel, label=str(cIdx))
        line_2, = ax2.plot(x, y_rotvel, label=str(cIdx))
        line_3, = ax3.plot(x, y_rotvel_x, label=str(cIdx))
        line_4, = ax4.plot(x, y_rotvel_y, label=str(cIdx))
        line_5, = ax5.plot(x, y_rotvel_z, label=str(cIdx))
        handle1.append(line_1)
        handle2.append(line_2)
        handle2.append(line_3)
        handle2.append(line_4)
        handle2.append(line_5)
        
    fig.tight_layout()
    
    # ax1.legend(handles=handle1)
    # ax2.legend(handles=handle2)



#===================================================================
def calc_hor_ver_from_vecs(a,b):
    """
    Use for vectors whose roll elements are too small
    """
    r = []
    return r
    
    
def calc_rot_matrix_list_from_vecs(a, b):
    """
    https://math.stackexchange.com/questions/180418/calculate-rotation-matrix-to-align-vector-a-to-vector-b-in-3d
    matrix R that rotates unit vector a onto unit vector b
    input: v1: (N,3)
    input: v2: (N,3)
    output: r: (N,3,3)
    """
    rot_list = []
    for f in range(a.shape[0]):                           
        v1 = a[f]
        v2 = b[f]
        v = np.cross(v1, v2) 
        s = np.linalg.norm(v)
        c = np.dot(v1, v2)
        vx = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])                           
        mat = np.eye(3) + vx + np.dot(vx,vx)/(1+c)
        rot_list.append(mat)
    r = np.array(rot_list)    
    return r


def calc_rot_omega_list(rot):
    """
    https://gamedev.stackexchange.com/questions/189950/calculate-angular-velocity-from-rotation-matrix-difference
    https://www.youtube.com/watch?app=desktop&v=szMaPkJEMrw
    input: rot: (N,3,3)
    output: omega: (N,3,3)
    """
    omega_list = []
    for f, r in enumerate(rot):
        inverse = scipy.linalg.inv(r + np.eye(3))
        omega = 2*np.eye(3) - 4*inverse
        omega = omega * 180 / np.pi
        omega_list.append(omega)
    return omega_list
    

def butter_lowpass(cutoff, fs, order=5):
    return butter(order, cutoff, fs=fs, btype='low', analog=False)

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y